In [ ]:
#tdiffusersenv
!pip install --upgrade diffusers transformers accelerate scipy safetensors

!pip install --upgrade diffusers[torch]
!pip install torch
!pip install numpy==1.26.4 --force-reinstall

In [ ]:
!pip install peft

In [ ]:
!python -c "import transformers; print(transformers.__version__)"
!python -c "import scipy; print(scipy.__version__)"
!python -c "import diffusers; print(diffusers.__version__)"
!python -c "import transformers; print(transformers.__version__)"
!python -c "import numpy; print(numpy.__version__)"
!python -c "import peft; print(peft.__version__)"
#4.49.0
#1.15.2
#0.32.2
#4.49.0
#1.26.4
#0.14.0

In [11]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
import gc
import time

model_id = "stabilityai/stable-diffusion-2-1"
lora_path = "tantanbei/lqq"
lora_path = "/Users/tankaide/ai/model_training/natalie_dormer_lora/model/nd-sd2-1-v1.safetensors"

# Use the Euler scheduler here instead
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler)
pipe = pipe.to("mps")  # macOS 需要 float32

# Recommended if your computer has < 64 GB of RAM
pipe.enable_attention_slicing()

# **加载 LoRA 权重**
pipe.load_lora_weights(lora_path)

# **应用 LoRA**
pipe.fuse_lora(scale=1)

# **3. 设置 LoRA 影响力**
#pipe.set_adapters(["default"])  # 选择适配的 LoRA
#pipe.set_adapters_weights([0.6])  # 设置权重（scale=0.6）

prompt = "girl with yellow hair on sunset"
image = pipe(prompt, width=768, height=768).images[0]

# image = image.convert("RGB")  
image.save("./output/astronaut_rides_horse.png")

del pipe
gc.collect()
torch.mps.empty_cache()  # 清理 Mac MPS 显存



100%|██████████| 50/50 [00:59<00:00,  1.19s/it]


In [9]:
#使用 diffusers 提供的转换脚本，将模型转换为 ComfyUI 支持的格式

!python diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py \
--model_path /Users/tankaide/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1/snapshots/5cae40e6a2745ae2b01ad92ae5043f95f23644d6 \
--checkpoint_path output/models--stabilityai--stable-diffusion-2-1.safetensors \
--use_safetensors

Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
